In [106]:
import pandas as pd
from finta import TA
from pathlib import Path

df_demo = pd.read_csv(Path('raw_data/SPY_data.csv'))
df_demo_demo = '../raw_data/SPY_data.csv'


In [107]:
def create_df(df_ohlcv):
    df = pd.read_csv(Path(f'raw_data/{df_ohlcv}'))
    df = df.drop(columns='Unnamed: 0')
    df.set_index('date', inplace=True)
    return df 

In [114]:
def choose_indicator(filepath, indicator_all_caps):
    df = create_df(filepath)
    if indicator_all_caps == 'ATR':
        df['ATR'] = TA.ATR(df)
        
    elif indicator_all_caps == 'BBANDS':
        df[['BBANDS Upper','Moving Average', 'BBands Lower']] = TA.BBANDS(df)

    elif indicator_all_caps == 'VBM':
        df['VBM'] = TA.VBM(df)

    elif indicator_all_caps == 'KAMA':
        df['KAMA'] = TA.VBM

    elif indicator_all_caps == 'DYMI':
        df['DYMI'] = TA.DYMI(df)

    elif indicator_all_caps == 'KC':
        df[['KC Upper', 'KC Lower']] = TA.KC(df)

    elif indicator_all_caps == 'APZ':
        df[['APZ Upper', 'APZ Lower']] = TA.APZ(df)
       
    elif indicator_all_caps == 'MI':
        df['MI'] = TA.MI(df)

    else:
        print("Indicator not a volatility indicator")
         

    return df

In [ ]:

def custom_pair_indicators(filepath, indicator_all_caps_1, indicator_all_caps_2):
    df_1 = choose_indicator(filepath, indicator_all_caps_1)
    df_2 = choose_indicator(filepath, indicator_all_caps_2)
    
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df

In [ ]:
def drop_any_column(df, *drop_list):
    for drop_me in drop_list:
        df = df.drop(columns=drop_me)
    return df


In [112]:
def mix_pair_indicators(df_1, df_2):
    df = df_1.merge(df_2, how='inner', right_index=True, left_index=True)
    df = df.drop(columns=['open_y', 'high_y','low_y','close_y', 'volume_y','trade_count_y','vwap_y'])
    df = df.rename(columns={'open_x':'open', 'high_x':'high', 'low_x':'low', 'close_x':'close', 'volume_x':'volume', 'trade_count_x':'trade_count', 'vwap_x':'vwap'})
    return df


In [141]:
frame = custom_pair_indicators(df_demo_demo,"MI", "APZ")

In [ ]:
def apply_rolling_window(df,*columns, period):
    for col in columns:
        df[f'{col}'] = df[f'{col}'].rolling(window=period).mean()
    return df


In [153]:
def get_column_range(df, *columns, period):
    # take a column and create a derivative range column with the desired period 
    for col in columns:
        high = df[f'{col}'].rolling(window=period).max()
        low = df[f'{col}'].rolling(window=period).min()
        df[f'{col} range'] = high - low
        
    return df

get_column_range(frame, 'close', period=5)    


,open,high,low,close,volume,trade_count,vwap,MI,APZ Upper,APZ Lower,close range
date,,,,,,,,,,,
2015-12-01 05:00:00+00:00,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410,NaN,214.100000,207.260000,NaN
2015-12-02 05:00:00+00:00,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,NaN,213.026939,205.023583,2.14
2015-12-03 05:00:00+00:00,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,NaN,211.608214,202.077730,2.96
2015-12-04 05:00:00+00:00,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,NaN,213.277980,202.660107,4.08
2015-12-07 05:00:00+00:00,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,NaN,213.531120,202.449012,1.39
...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24 04:00:00+00:00,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,24.325757,455.807927,419.932725,6.68
2022-03-25 04:00:00+00:00,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,24.272693,458.139062,422.805509,2.21
2022-03-28 04:00:00+00:00,452.04,455.9100,450.0600,455.91,68356914,620868,453.294974,24.238979,460.676124,425.888839,3.22
